# Sharpness-Aware Private Aggregation of Teacher Ensembles (Sharp-PATE)

## Step 1: Load the data

In [1]:
import torch
from torchvision import datasets, transforms
from torch.utils.data import Subset
from sam import SAM

# Transform the image to a tensor and normalize it
transform = transforms.Compose([transforms.ToTensor(),
                                transforms.Normalize((0.5,), (0.5,))])

# Load the train and test data by using the transform
train_data = datasets.MNIST(root='data', train=True, download=True, transform=transform)
test_data = datasets.MNIST(root='data', train=False, download=True, transform=transform)

  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting data/MNIST/raw/train-images-idx3-ubyte.gz to data/MNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting data/MNIST/raw/train-labels-idx1-ubyte.gz to data/MNIST/raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting data/MNIST/raw/t10k-images-idx3-ubyte.gz to data/MNIST/raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting data/MNIST/raw/t10k-labels-idx1-ubyte.gz to data/MNIST/raw



In [2]:
num_teachers = 100 # Define the num of teachers
batch_size = 32 # Teacher batch size

def get_data_loaders(train_data, num_teachers):
    """ Function to create data loaders for the Teacher classifier """
    teacher_loaders = []
    data_size = len(train_data) // num_teachers
    
    for i in range(data_size):
        indices = list(range(i*data_size, (i+1)*data_size))
        subset_data = Subset(train_data, indices)
        loader = torch.utils.data.DataLoader(subset_data, batch_size=batch_size)
        teacher_loaders.append(loader)
        
    return teacher_loaders

teacher_loaders = get_data_loaders(train_data, num_teachers)

In [3]:
# Create the public dataset by using 90% of the Test data as train data and remaining
# 10% as test data.
student_train_data = Subset(test_data, list(range(9000)))
student_test_data = Subset(test_data, list(range(9000, 10000)))

student_train_loader = torch.utils.data.DataLoader(student_train_data, batch_size=batch_size)
student_test_loader = torch.utils.data.DataLoader(student_test_data, batch_size=batch_size)

## Step 2: Define the architecture to train the teacher models

In [4]:
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

class Classifier(nn.Module):
    """ A Simple Feed Forward Neural Network. We use 2 convolutional layers followed by 2 fullu-connected layers
    """
    def __init__(self):
        super().__init__()
        
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)
    
    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(-1, 320)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        return F.log_softmax(x)

## Step 3: Define train and prediction functions

In [5]:
def train(model, trainloader, criterion, optimizer, epochs=10):
    """ This function trains a single Classifier model """
    running_loss = 0
    for e in range(epochs):
        model.train()
        
        for images, labels in trainloader:
            optimizer.zero_grad()
            output = model.forward(images)
            loss = criterion(output, labels)
            loss.backward()
            optimizer.first_step(zero_grad=True)
            # optimizer.step()
            criterion(model.forward(images), labels).backward()
            optimizer.second_step(zero_grad=True)
            
            running_loss += loss.item()
        

In [6]:
def predict(model, dataloader):
    """ This function predicts labels for a dataset 
        given the model and dataloader as inputs. 
    """
    outputs = torch.zeros(0, dtype=torch.long)
    model.eval()
    
    for images, labels in dataloader:
        output = model.forward(images)
        ps = torch.argmax(torch.exp(output), dim=1)
        outputs = torch.cat((outputs, ps))
        
    return outputs

In [7]:
def train_models(num_teachers, rho_val):
    """ Trains *num_teacher* models (num_teachers being the number of teacher classifiers) """
    models = []
    for i in range(num_teachers):
        model = Classifier()
        criterion = nn.NLLLoss()
        base_optimizer = optim.Adam
        optimizer = SAM(model.parameters(), base_optimizer, rho=rho_val, adaptive=True, lr=0.003)
        train(model, teacher_loaders[i], criterion, optimizer)
        models.append(model)
    return models

##use rho=0.0005 (found using grid search)
models = train_models(num_teachers, 0.0005)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:25: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


## Step 4: Create the aggregated teacher ensemble to generate student labels

In [8]:
import numpy as np

epsilon = 0.2
def aggregated_teacher(models, dataloader, epsilon):
    """ Take predictions from individual teacher model and 
        creates the true labels for the student after adding 
        laplacian noise to them 
    """
    preds = torch.torch.zeros((len(models), 9000), dtype=torch.long)
    for i, model in enumerate(models):
        results = predict(model, dataloader)
        preds[i] = results
    
    labels = np.array([]).astype(int)
    for image_preds in np.transpose(preds):
        label_counts = np.bincount(image_preds, minlength=10)
        beta = 1 / epsilon

        for i in range(len(label_counts)):
            label_counts[i] += np.random.laplace(0, beta, 1)

        new_label = np.argmax(label_counts)
        labels = np.append(labels, new_label)
    
    return preds.numpy(), labels

teacher_models = models
preds, student_labels = aggregated_teacher(teacher_models, student_train_loader, epsilon)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:25: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


## Step 5: Train the student model using student labels obtained from the Step-4

In [9]:
def student_loader(student_train_loader, labels):
    for i, (data, _) in enumerate(iter(student_train_loader)):
        yield data, torch.from_numpy(labels[i*len(data): (i+1)*len(data)])

student_model = Classifier()
criterion = nn.NLLLoss()
base_optimizer = optim.Adam

##use rho=0.05 (found using grid search)
optimizer = SAM(student_model.parameters(), base_optimizer, 0.05, adaptive=True, lr=0.003)
epochs = 10
steps = 0
running_loss = 0
for e in range(epochs):
    student_model.train()
    train_loader = student_loader(student_train_loader, student_labels)

    for images, labels in train_loader:
        steps += 1
        
        optimizer.zero_grad()
        output = student_model.forward(images)
        loss = criterion(output, labels)
        loss.backward()
        optimizer.first_step(zero_grad=True)
        # optimizer.step()
        criterion(student_model.forward(images), labels).backward() 
        optimizer.second_step(zero_grad=True)

        running_loss += loss.item()
        
        if steps % 50 == 0:
            test_loss = 0
            accuracy = 0
            student_model.eval()
            with torch.no_grad():
                for images, labels in student_test_loader:
                    log_ps = student_model(images)
                    test_loss += criterion(log_ps, labels).item()
                    
                    # Accuracy
                    ps = torch.exp(log_ps)
                    top_p, top_class = ps.topk(1, dim=1)
                    equals = top_class == labels.view(*top_class.shape)
                    accuracy += torch.mean(equals.type(torch.FloatTensor))
            student_model.train()
            print("Epoch: {}/{}.. ".format(e+1, epochs),
                  "Training Loss: {:.3f}.. ".format(running_loss/len(student_train_loader)),
                  "Test Loss: {:.3f}.. ".format(test_loss/len(student_test_loader)),
                  "Test Accuracy: {:.3f}".format(accuracy/len(student_test_loader)))
            running_loss = 0

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:25: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


Epoch: 1/10..  Training Loss: 0.334..  Test Loss: 0.845..  Test Accuracy: 0.787
Epoch: 1/10..  Training Loss: 0.160..  Test Loss: 0.522..  Test Accuracy: 0.853
Epoch: 1/10..  Training Loss: 0.120..  Test Loss: 0.335..  Test Accuracy: 0.913
Epoch: 1/10..  Training Loss: 0.083..  Test Loss: 0.351..  Test Accuracy: 0.899
Epoch: 1/10..  Training Loss: 0.088..  Test Loss: 0.254..  Test Accuracy: 0.936
Epoch: 2/10..  Training Loss: 0.131..  Test Loss: 0.307..  Test Accuracy: 0.920
Epoch: 2/10..  Training Loss: 0.088..  Test Loss: 0.240..  Test Accuracy: 0.930
Epoch: 2/10..  Training Loss: 0.074..  Test Loss: 0.242..  Test Accuracy: 0.929
Epoch: 2/10..  Training Loss: 0.068..  Test Loss: 0.227..  Test Accuracy: 0.941
Epoch: 2/10..  Training Loss: 0.054..  Test Loss: 0.227..  Test Accuracy: 0.939
Epoch: 2/10..  Training Loss: 0.062..  Test Loss: 0.266..  Test Accuracy: 0.936
Epoch: 3/10..  Training Loss: 0.105..  Test Loss: 0.221..  Test Accuracy: 0.938
Epoch: 3/10..  Training Loss: 0.068..  T